In [ ]:
from train import train
from utils import *

d:\main\fyp\myfyp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
convert_dataset_and_save_as_file("train.json", "train.jsonl")
train(model_name="huggyllama/llama-7b", dataset_path= "train.jsonl")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
Loading checkpoint shards: 100%|██████████| 2/2 [00:28<00:00, 14.23s/it]


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622


Map: 100%|██████████| 33/33 [00:00<00:00, 616.88 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
d:\main\fyp\myfyp\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


d:\main\fyp\myfyp\.venv\Lib\site-packages\peft\utils\other.py:1107: UserWarning: Unable to fetch remote file due to the following error 401 Client Error: Unauthorized for url: https://huggingface.co/huggyllama/llama-7b/resolve/main/config.json (Request ID: Root=1-67f238b9-226d6461281163610dcbf0e9;948967fa-0faa-4567-b99a-6cf0c4282ade)

Invalid credentials in Authorization header - silently ignoring the lookup for the file config.json in huggyllama/llama-7b.
  warnings.warn(
d:\main\fyp\myfyp\.venv\Lib\site-packages\peft\utils\save_and_load.py:236: UserWarning: Could not find a config file in huggyllama/llama-7b - will assume that the vocabulary was not modified.
  warnings.warn(
d:\main\fyp\myfyp\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the cu

KeyboardInterrupt: 